In [2]:
# Import necessary libraries
import json
from web3 import Web3
from solcx import compile_standard, install_solc
import solcx

# Connect to the local Ethereum network (Ganache)
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))

# Check if connection is successful
if not w3.is_connected():
    raise Exception("Failed to connect to the Ethereum network")




In [3]:
# Set the default account (deployer)
w3.eth.default_account = w3.eth.accounts[0]
print(w3.eth.default_account)




0xd41d88454EF0C2A81b7d2B1229E2aAC779672721


In [4]:
# Compile the contract using solcx
solcx.install_solc('0.8.0')
solcx.set_solc_version('0.8.0')

# Path to your Solidity contract
contract_path = "./ProofOfReputation.sol"
  # Update this path to your contract

# Compile the Solidity contract
with open(contract_path, "r") as file:
    contract_source_code = file.read()

compiled_sol = solcx.compile_standard({
    "language": "Solidity",
    "sources": {
        "ProofOfReputation.sol": {
            "content": contract_source_code
        }
    },
    "settings": {
        "outputSelection": {
            "*": {
                "*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]
            }
        }
    }
})

# Extract ABI and bytecode
abi = compiled_sol['contracts']['ProofOfReputation.sol']['ProofOfReputation']['abi']
bytecode = compiled_sol['contracts']['ProofOfReputation.sol']['ProofOfReputation']['evm']['bytecode']['object']


In [5]:
# Deploy the contract
ProofOfReputation = w3.eth.contract(abi=abi, bytecode=bytecode)

# Deploy the contract and get the transaction hash
tx_hash = ProofOfReputation.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
contract_address = tx_receipt.contractAddress
print(f"Contract deployed at address: {contract_address}")


Contract deployed at address: 0xC3C5157A27c4e419681d8901f2A10318aD6F6dce


In [6]:
# Get the deployed contract instance
deployed_contract = w3.eth.contract(address=contract_address, abi=abi)

# Function to log gas used
def log_gas_usage(tx_hash, action):
    receipt = w3.eth.get_transaction_receipt(tx_hash)
    print(f"Gas used for {action}: {receipt.gasUsed}")

# Add a new client
def add_client(client_address, initial_reputation):
    tx_hash = deployed_contract.functions.addClient(client_address, initial_reputation).transact()
    log_gas_usage(tx_hash, f"Adding client {client_address}")

# Submit a job
def submit_job(client_address, job_result):
    tx_hash = deployed_contract.functions.submitJob(job_result).transact({'from': client_address})
    log_gas_usage(tx_hash, f"Client {client_address} submitting job")

# Validate all jobs
def validate_all_jobs(validator_address):
    tx_hash = deployed_contract.functions.validateAllJobs().transact({'from': validator_address})
    log_gas_usage(tx_hash, "Validating all jobs")

# Update weights
def update_weights(weightH, weightT, weightC, weightP, weightV):
    tx_hash = deployed_contract.functions.updateWeights(weightH, weightT, weightC, weightP, weightV).transact()
    log_gas_usage(tx_hash, "Updating weights")

# Example usage
print("Adding clients...")
add_client(w3.eth.accounts[1], 100)
add_client(w3.eth.accounts[2], 200)

print("Submitting jobs...")
submit_job(w3.eth.accounts[1], "Job 1 result")
submit_job(w3.eth.accounts[2], "Job 2 result")

print("Validating jobs...")
current_validator = deployed_contract.functions.getCurrentValidator().call()
validate_all_jobs(current_validator)

print("Updating reputation weights...")
update_weights(10, 30, 20, 20, 20)

# Fetching updated reputations
def get_reputation(client_address):
    reputation = deployed_contract.functions.getReputationScore(client_address).call()
    print(f"Reputation of client {client_address}: {reputation}")

print("Fetching reputations...")
get_reputation(w3.eth.accounts[1])
get_reputation(w3.eth.accounts[2])

Adding clients...
Gas used for Adding client 0xc7c81e3F14829c1E7b4bDf8c118D12504f9fD7Fa: 174395
Gas used for Adding client 0xaA739F8Bd0014fa6d83cB807944830782bA6940F: 129398
Submitting jobs...
Gas used for Client 0xc7c81e3F14829c1E7b4bDf8c118D12504f9fD7Fa submitting job: 129668
Gas used for Client 0xaA739F8Bd0014fa6d83cB807944830782bA6940F submitting job: 95468
Validating jobs...
Gas used for Validating all jobs: 153473
Updating reputation weights...
Gas used for Updating weights: 51292
Fetching reputations...
Reputation of client 0xc7c81e3F14829c1E7b4bDf8c118D12504f9fD7Fa: 0
Reputation of client 0xaA739F8Bd0014fa6d83cB807944830782bA6940F: 3
